# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
api_cities_file = '../WeatherPY/cities.csv'
weather_df = pd.read_csv(api_cities_file, encoding="utf-8")
weather_df


,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Poum,-20.2333,164.0167,78.75,82,86,14.79,NC,1636773886
1,Touros,-5.1989,-35.4608,79.70,80,98,12.84,BR,1636773888
2,Ushuaia,-54.8000,-68.3000,44.26,87,20,8.05,AR,1636773888
3,Laguna,38.4210,-121.4238,64.24,96,90,3.00,US,1636773889
4,Saint-Philippe,-21.3585,55.7679,79.48,76,50,9.44,RE,1636773889
...,...,...,...,...,...,...,...,...,...
538,Daru,-9.0763,143.2092,87.60,62,76,8.61,PG,1636774135
539,Severnyy,50.6770,36.5532,33.84,93,100,8.95,RU,1636774135
540,Alghero,40.5589,8.3181,50.09,100,0,8.05,IT,1636774136
541,Umm Kaddadah,13.6017,26.6876,74.50,18,0,13.56,SD,1636774136


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Google API Key
gmaps.configure(api_key=g_key)
weather_df = weather_df.rename({'Latitude': 'Lat', 'Longitude': 'Lng'}, axis=1)

In [4]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)

In [5]:
# Add Heatmap layer to map.
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_df = weather_df.loc[(weather_df["Max Temperature (F)"]>=70)&(weather_df["Max Temperature (F)"]<=90)&(weather_df["Humidity"]>=30)&(weather_df["Humidity"]<=50)]
ideal_df.dropna(inplace = True)
ideal_df

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date
12,Atar,20.5169,-13.0499,77.79,34,65,1.88,MR,1636773892
32,Busselton,-33.6500,115.3333,70.05,41,14,10.60,AU,1636773901
109,Sur,22.5667,59.5289,77.59,49,0,11.43,OM,1636773936
166,Mackay,-21.1500,149.2000,85.05,44,38,12.66,AU,1636773964
198,Hong Kong,22.2855,114.1577,75.96,46,99,1.99,HK,1636773979
252,La Palma,33.8464,-118.0467,84.20,35,1,0.00,US,1636774005
324,Gold Coast,-28.0000,153.4333,82.24,34,0,14.00,AU,1636774038
348,Taree,-31.9000,152.4667,73.26,33,40,24.16,AU,1636774048
363,Tāndā,26.5500,82.6500,70.99,50,5,0.98,IN,1636774055
409,Mundra,22.8500,69.7333,78.28,34,76,7.58,IN,1636774077


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_df
hotel_df["Hotel Name"]= ""
hotel_df["Hotel Address"]= ""
hotel_df

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Lat,Lng,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
12,Atar,20.5169,-13.0499,77.79,34,65,1.88,MR,1636773892,,
32,Busselton,-33.6500,115.3333,70.05,41,14,10.60,AU,1636773901,,
109,Sur,22.5667,59.5289,77.59,49,0,11.43,OM,1636773936,,
166,Mackay,-21.1500,149.2000,85.05,44,38,12.66,AU,1636773964,,
198,Hong Kong,22.2855,114.1577,75.96,46,99,1.99,HK,1636773979,,
252,La Palma,33.8464,-118.0467,84.20,35,1,0.00,US,1636774005,,
324,Gold Coast,-28.0000,153.4333,82.24,34,0,14.00,AU,1636774038,,
348,Taree,-31.9000,152.4667,73.26,33,40,24.16,AU,1636774048,,
363,Tāndā,26.5500,82.6500,70.99,50,5,0.98,IN,1636774055,,
409,Mundra,22.8500,69.7333,78.28,34,76,7.58,IN,1636774077,,


In [8]:
#itterate 
locations

,Lat,Lng
0,-20.2333,164.0167
1,-5.1989,-35.4608
2,-54.8000,-68.3000
3,38.4210,-121.4238
4,-21.3585,55.7679
...,...,...
538,-9.0763,143.2092
539,50.6770,36.5532
540,40.5589,8.3181
541,13.6017,26.6876


In [22]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_radius = 5000
hotel_distance = "distance"
hotel_type = "lodging"

params = {
        "radius": hotel_radius,
        "types": hotel_type,
        "key": g_key,
        }
# params = {
#     "keyword":"hotel",
#     "radius": hotel_radius,
#     "rankby": hotel_distance,
#     "type": hotel_type,
#     "key": g_key,
# }
#itterate through df. append locations to params.  ##goes inside for loop

for i, r in hotel_df.iterrows():

    lat=r["Lat"]
    lng=r["Lng"]
    params["location"] = f"{lat},{lng}"

    response = requests.get(url, params=params)
    hotel_json = response.json()
    try: 
#         pprint.pprint(hotel_json)
#         pprint.pprint([hotel_json["results"][0]["name"]])
#         pprint.pprint([hotel_json["results"][0]["vicinity"]])
        hotel_df.loc[i, "Hotel Name"]= [hotel_json["results"][0]["name"]]
        hotel_df.loc[i, "Hotel Address"]= [hotel_json["results"][0]["vicinity"]]
#         for x in hotel_json:
#         hotel_df["Hotel Name"].append(name)
#         hotel_df["Hotel Address"].append(address)
    except (KeyError, ValueError):
        print(f"Not appending params")
    print("------------")



/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------


In [23]:
hotel_df

,City,Lat,Lng,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
12,Atar,20.5169,-13.0499,77.79,34,65,1.88,MR,1636773892,Oumou Elghoura,"Mbarka Oumara Derriere l'hopital regional, Atar"
32,Busselton,-33.6500,115.3333,70.05,41,14,10.60,AU,1636773901,Observatory Guest House,"7 Brown Street, Busselton"
109,Sur,22.5667,59.5289,77.59,49,0,11.43,OM,1636773936,Sur Plaza Hotel,Oman
166,Mackay,-21.1500,149.2000,85.05,44,38,12.66,AU,1636773964,International Lodge Motel,"40 MacAlister Street, Mackay"
198,Hong Kong,22.2855,114.1577,75.96,46,99,1.99,HK,1636773979,Mini Hotel Central,"38 Ice House Street, Central"
252,La Palma,33.8464,-118.0467,84.20,35,1,0.00,US,1636774005,La Quinta Inn & Suites by Wyndham Buena Park,"3 Centerpointe Drive, La Palma"
324,Gold Coast,-28.0000,153.4333,82.24,34,0,14.00,AU,1636774038,Mantra Towers of Chevron Surfers Paradise,"23 Ferny Avenue, Surfers Paradise"
348,Taree,-31.9000,152.4667,73.26,33,40,24.16,AU,1636774048,All Seasons Country Lodge Taree,"110 Manning River Drive, Taree"
363,Tāndā,26.5500,82.6500,70.99,50,5,0.98,IN,1636774055,MUMTAZ GANJ TANDA,"Tanda - Kalwari Road, Chhajjapur, Tanda"
409,Mundra,22.8500,69.7333,78.28,34,76,7.58,IN,1636774077,Hotel Bansidhar Palace,"Shakti Nagar, Adani Road, Mundra"


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
# Create bank symbol layer
marker_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
# Create a combined map
fig = gmaps.figure(map_type="TERRAIN")

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))